In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

c:\Users\Win 11\Desktop\RAG Pipeline\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [5]:
# Step 1: Load and chunk the document
loader = TextLoader("langchain_rag_dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)
chunks

[Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain is an open-source framework designed to simplify the development of applications using large language models (LLMs).\nLangChain provides abstractions for working with prompts, chains, memory, and agents, making it easier to build complex LLM-based systems.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='The framework supports integration with various vector databases like FAISS and Chroma for semantic retrieval.\nLangChain enables Retrieval-Augmented Generation (RAG) by allowing developers to fetch relevant context before generating responses.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
 Document(metadata={'

In [6]:
# Step 2: FAISS Vector Store with HuggingFace Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)

In [7]:
### Step 3: Create MMR Retriever
retriever=vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3}
)

In [8]:
# Step 4: Prompt and LLM
prompt = PromptTemplate.from_template("""
    Answer the question based on the context provided.
    Context:
    {context}
    Question: {input}
    """)
llm=init_chat_model("openai:gpt-3.5-turbo")

In [9]:
# Step 5: RAG Pipeline
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=document_chain)

In [11]:
# Step 6: Query
query = {"input": "How does LangChain support agents and memory?"}
response = rag_chain.invoke(query)

print("✅ Answer:\\n", response["answer"])
response

✅ Answer:\n LangChain supports agents by allowing them to interact with external APIs, databases, and use tools like calculators and custom functions based on instructions. It also supports memory through features like ConversationBufferMemory for conversational memory and ConversationSummaryMemory for summarization memory. This helps LLM-powered applications retain previous interactions and make multi-turn conversations more coherent. LangChain also allows LLMs to act as agents that decide which tool to call and in what order during a task, further enhancing their capabilities.


{'input': 'How does LangChain support agents and memory?',
 'context': [Document(id='76f42bfa-1797-4c1b-94b8-6fe5e5f52a8b', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
  Document(id='ced045a2-5088-4af0-87d1-15975412894f', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain agents can interact with external APIs and databases, enhancing the capabilities of LLM-powered applications.\nRAG pipelines in LangChain involve document loading, splitting, embedding, retrieval, and LLM-based response generation.'),
  Document(id='9be80738-b7ea-47cd-81ab-e64f102d02bf', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain allows LLMs to act as agents that decide which tool to call and in what order duri